In [1]:
import time
import requests
from io import BytesIO
from PIL import Image
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from dotenv import load_dotenv
import requests
import os
import re
import matplotlib.pyplot as plt
import json

load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")

# Importar librerias de websracpping

In [2]:
def extraer_info(url):
    options = Options()
    options.add_argument("--headless")  # Ejecutar en modo headless para que no se aprecie la ventana
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    service = Service()  # Crear el servicio para el controlador
    driver = webdriver.Chrome(service=service, options=options)  # Iniciar el navegador con opciones
    driver.get(url)
    time.sleep(5)  # Espera a que cargue la página y JS

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    titles = []
    descriptions = []
    image_urls = []
    gif_urls = []

    # Extraer todos los títulos y subtítulos (h1 a h6)
    for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        titles.append(heading.get_text(strip=True))

    # Extraer descripciones desde etiquetas <p> o <div>
    for paragraph in soup.find_all('p'):
        text = paragraph.get_text(strip=True)
        if len(text) > 30:  # Filtrar textos cortos
            descriptions.append(text)

    # Extraer imágenes y GIFs
    blacklist_keywords = [
        'logo', 'loggo', 'logotipo', 'favicon', 'icon', 'ico', 'brandmark',
        'marca', 'log_', '_logo', 'log-', '-logo', 'logggo', 'watermark',
        'paypal', 'sistecredito', 'visa', 'mastercard', 'payment', 'pago',
        'credit', 'debit', 'secure', 'checkout', 'cart'
    ]

    for img in soup.find_all('img'):
        src = img.get('src') or img.get('data-src') or img.get('data-image')
        if src:
            full_url = urljoin(url, src)
            full_url_lower = full_url.lower()

            # Excluir imágenes irrelevantes
            if any(keyword in full_url_lower for keyword in blacklist_keywords):
                continue

            # Clasificar imágenes como GIFs o imágenes normales
            if "gif" in full_url_lower:
                gif_urls.append(full_url)
            else:
                image_urls.append(full_url)

    # Eliminar duplicados
    image_urls = list(set(image_urls))
    gif_urls = list(set(gif_urls))

    # Filtrar imágenes por tamaño (opcional, requiere requests y Pillow)
    filtered_image_urls = []
    for img_url in image_urls:
        try:
            response = requests.get(img_url, stream=True, timeout=5)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                width, height = img.size
                # Excluir imágenes pequeñas (por ejemplo, iconos)
                if width > 100 and height > 100:  # Ajusta el tamaño mínimo según sea necesario
                    filtered_image_urls.append(img_url)
        except Exception:
            continue

    # Agregar todos los títulos como un valor adicional
    todos_los_titulos = titles
    return titles, descriptions, todos_los_titulos, filtered_image_urls, gif_urls

In [4]:
url = "https://dyshopcol.com/products/parches-de-ojos-durazno-bioaqua"
titulo, descripcion, todos_los_titulos, imagenes, gifs = extraer_info(url)


# Mostrar imágenes y URLs
for img_url in imagenes:
    print("URL:", img_url)
    response = requests.get(img_url)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        print(f"Error al cargar la imagen desde {img_url}")

NameError: name 'extraer_info' is not defined

## Agente ia para generar titulos y copies


In [3]:
product_url = input("Ingrese la URL del producto: ").strip()
titulo, descripciones, todos_los_titulos, filtered_image_urls, gif_urls = extraer_info(product_url)

In [4]:
print("Descripciones de productos:", descripciones)
print("Todos los títulos:", todos_los_titulos)
print("URLs de imágenes filtradas:", filtered_image_urls)
print("URLs de GIFs:", gif_urls)

Descripciones de productos: ['ENVÍO GRATIS + PAGO CONTRAENTREGA', 'Desde elprimer usonotarás una sensación de frescura y descongestión.Con uso constante, en pocos días verás unareducción visible de ojeras, bolsitas y arrugas.', '¡Sí! Nuestros parches están formulados coningredientes suaves y naturales, libres de alcohol y parabenos. Son ideales parapiel sensibley pueden usarse a cualquier edad.', '¡Totalmente seguro! Solo pagascuando recibes el producto en tu casa. No necesitas dar datos bancarios.Sin riesgos, sin complicaciones.', 'Cuando pedí la verdad no tenía muchas esperanzas, lo pedí solo por curiosidad, pero la verdad, estoy gratamente sorprendida, los he usado y realmente he obtenido muy buenos resultados, les seguiré comprando!', 'Gracias por tan excelente calidad y por ofrecer la facilidad de pago en casa, excelente producto', 'me llegó hace 2 semanas y los resultados hasta ahora muy buenos!!', 'Me encanta!! ha reducido muchisimo mis lineas de expresión y ojeras', 'Lo probaré

In [5]:
class AgenteTitulosYCopies:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv("DEEPSEEK_API_KEY")
        self.generated_file = "contenido_generado.json"

    def _clean_text(self, text_data):
        """Limpia texto eliminando emojis y caracteres no deseados"""
        if isinstance(text_data, list):
            text_data = " ".join(str(item) for item in text_data if item)
        # Eliminar emojis y caracteres especiales excepto puntuación básica
        text_data = re.sub(r'[^\w\s.,;!?¿¡áéíóúÁÉÍÓÚñÑ-]', '', str(text_data))
        return re.sub(r'\s+', ' ', text_data).strip()

    def _call_ai(self, prompt):
        """Interacción con la API"""
        response = requests.post(
            "https://api.deepseek.com/chat/completions",
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self.api_key}"
            },
            json={
                "model": "deepseek-chat",
                "messages": [{
                    "role": "system",
                    "content": "Eres un redactor profesional de contenido comercial, que crea titulos y copies llamativos de venta eres todo un exito y capaz de vender hasta un dulce. No uses emojis ni caracteres especiales."
                }, {
                    "role": "user",
                    "content": prompt
                }],
                "temperature": 0.7
            }
        )
        if response.status_code != 200:
            raise Exception(f"Error de API: {response.status_code} - {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def generate_results(self, titulo, descripciones):
        """Genera contenido sin mostrar procesos intermedios"""
        # Limpieza de datos
        clean_title = self._clean_text(next((t for t in titulo if t and len(t) > 10), ""))
        clean_desc = self._clean_text(" ".join(d for d in descripciones if d and len(d) > 30))

        if not clean_title or not clean_desc:
            raise ValueError("No se pudo obtener información válida del producto")

        # Generar títulos y copies
        content_prompt = f"""
        Genera 2 títulos publicitarios y 2 copies profesionales, ganadores, llamativos, super redactados como un HUMANO experto en el tema para redes sociales basados en:

        PRODUCTO: {clean_title}
        DESCRIPCIÓN: {clean_desc}

        REQUISITOS:
        - Sin emojis ni caracteres especiales
        - Títulos máximo 50 caracteres
        - Copies de 2-3 frases persuasivas y que tengan los dos minimo 100 caracteres y maximo 248 caracteres
        - No usar palabras como "compra", "adquiere", "obtén" o similares   
        - Lenguaje profesional y orientado a beneficios reales
        - No inventar marcas o nombres comerciales
        - Incluir beneficios clave, ingredientes que tenga el producto y facilidad de pago

        FORMATO EXACTO:
        TITULO 1: [texto]
        TITULO 2: [texto]
        COPY 1: [texto]
        COPY 2: [texto]
        """
        generated_content = self._call_ai(content_prompt)

        # Procesar títulos y copies
        titles = []
        copies = []
        for line in generated_content.split('\n'):
            line = line.strip()
            if line.startswith('TITULO'):
                titles.append(line.split(': ')[1])
            elif line.startswith('COPY'):
                copies.append(line.split(': ')[1])

        # Generar públicos objetivos
        audiences_prompt = f"""
        Genera 5 públicos objetivos numerados (1-5) para este producto:

        PRODUCTO: {clean_title}
        CARACTERÍSTICAS: {clean_desc}

        REQUISITOS:
        - Sin emojis ni caracteres especiales
        - Cada público en una línea numerada
        - Que sea general y una oracion, maximo 20 caracteres
        - Basado en datos demográficos y comportamientos reales
        - Incluir preocupaciones específicas como piel sensible, estrés laboral, etc.
        - Evita hablar de publicos que prefieren pagos contra entrega o evitar datos bancarios, ventas online, etc.
        FORMATO:
        1. Público objetivo 1
        2. Público objetivo 2
        ...
        5. Público objetivo 5
        """
        audiences_response = self._call_ai(audiences_prompt)
        audiences = [re.sub(r'^\d+\.\s*', '', line.strip()) for line in audiences_response.split('\n') if re.match(r'^\d+\.', line.strip())]

        # Guardar resultados
        with open(self.generated_file, 'w', encoding='utf-8') as f:
            json.dump({
                "titulos": titles,
                "copies": copies,
                "publicos_objetivos": audiences
            }, f, ensure_ascii=False, indent=2)

        # Mostrar resultados limpios
        print("\nTITULOS GENERADOS:")
        for i, title in enumerate(titles, 1):
            print(f"{i}. {title}")

        print("\nCOPIES PARA REDES:")
        for i, copy in enumerate(copies, 1):
            print(f"{i}. {copy}")

        print("\nPUBLICOS OBJETIVOS:")
        for i, audience in enumerate(audiences, 1):  
            print(f"{i}. {audience}")

        return titles, copies, audiences


In [6]:
generator = AgenteTitulosYCopies()
generator.generate_results(todos_los_titulos, descripciones)


TITULOS GENERADOS:
1. Descubre el secreto para una mirada rejuvenecida
2. Adiós ojeras y arrugas con resultados visibles

COPIES PARA REDES:
1. Desde la primera aplicación sentirás una frescura única mientras los ingredientes naturales trabajan en tu piel. Formulado sin alcohol ni parabenos, es ideal incluso para pieles sensibles. Olvídate de complicaciones con nuestro pago contra entrega y envío gratis.
2. La fórmula suave pero poderosa de estos parches reduce ojeras, bolsas y líneas de expresión en días. Sin riesgos, sin datos bancarios, solo resultados reales que notarás rápidamente. Pruébalo y descubre por qué se ha convertido en el favorito de quienes buscan una mirada joven y descansada.

PUBLICOS OBJETIVOS:
1. Mujeres 30+ con ojeras marcadas
2. Hombres estresados con bolsas oculares
3. Piel sensible busca solucion natural
4. Adultos con lineas de expresion
5. Jovenes con falta de sueño cronico


(['Descubre el secreto para una mirada rejuvenecida',
  'Adiós ojeras y arrugas con resultados visibles'],
 ['Desde la primera aplicación sentirás una frescura única mientras los ingredientes naturales trabajan en tu piel. Formulado sin alcohol ni parabenos, es ideal incluso para pieles sensibles. Olvídate de complicaciones con nuestro pago contra entrega y envío gratis.',
  'La fórmula suave pero poderosa de estos parches reduce ojeras, bolsas y líneas de expresión en días. Sin riesgos, sin datos bancarios, solo resultados reales que notarás rápidamente. Pruébalo y descubre por qué se ha convertido en el favorito de quienes buscan una mirada joven y descansada.'],
 ['Mujeres 30+ con ojeras marcadas',
  'Hombres estresados con bolsas oculares',
  'Piel sensible busca solucion natural',
  'Adultos con lineas de expresion',
  'Jovenes con falta de sueño cronico'])

## agente ia que generar guiones

In [7]:
class AgenteGuiones:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv("DEEPSEEK_API_KEY")
        if not self.api_key:
            raise ValueError("No se encontró la API Key de DeepSeek en el archivo .env")

    def generate_script(self, titulo, descripcion, pais, publico_objetivo, angulo_de_venta, tono, tipo_de_video, duracion):
        prompt = f"""Crea un guión para un {tipo_de_video} de {duracion} segundos que promocione el siguiente producto:
            Título del producto: {titulo}
            Descripción del producto: {descripcion}
            que sera para un el pais de: {pais}
            con un Público objetivo de: {publico_objetivo}
            Ángulo de venta: {angulo_de_venta}
            Tono del video: {tono}

            Devuélveme el guión con la siguiente estructura de ejemplo y las siguientes instrucciones:
            1. No incluyas tiempos ni escenas, solo texto hablado.
            2. “Solo diálogo hablado para video publicitario”.
            3. Solo dame el guion y no tiene que tener encabezado de titulo ni nada, solo el texto.
            4. No incluyas emojis ni caracteres especiales.
            5. No incluyas información adicional, solo el guión.
            
            (Solo es un ejemplo no lo uses en el guion y no lo incluyas en el guion, se creativo y no uses esto como base, solo es un ejemplo de lo que quiero):
            Ejemplo de guión:
            ¿Tu mirada delata el cansancio y el paso del tiempo?
            Descubre los parches que revierten el tiempo en tu mirada.
            Con ingredientes suaves y libres de alcohol, ideales para pieles sensibles.
            Reducen bolsas, ojeras y arrugas desde la primera aplicación.
            Dale a tu piel el cuidado natural y efectivo que merece.
            Y lo mejor: paga solo cuando lo tengas en tus manos, sin riesgos.
            Fácil de usar, resultados visibles.
            Aumenta tu belleza natural sin esfuerzo.
            ¡Pídelos ya y devuélvele la frescura a tu mirada!
            Disponible en todo Colombia.
            """
        messages = [
            {"role": "system", "content": "Eres un guionista experto en videos cortos para publicidad."},
            {"role": "user", "content": prompt}
        ]

        response = requests.post(
            "https://api.deepseek.com/chat/completions",
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {self.api_key}"
            },
            json={
                "model": "deepseek-chat",
                "messages": messages,
                "temperature": 0.7
            }
        )

        if response.status_code != 200:
            raise Exception(f"Error al llamar la API de DeepSeek: {response.text}")

        return response.json()["choices"][0]["message"]["content"].strip()

In [8]:
import json
# Cargar el contenido del archivo JSON generado
with open("./contenido_generado.json", "r", encoding="utf-8") as file:
    contenido_generado = json.load(file)

# Extraer información relevante del JSON
titulos = contenido_generado.get("titulos", [])
copies = contenido_generado.get("copies", [])
publicos_objetivos = contenido_generado.get("publicos_objetivos", [])

In [9]:
pais = "Colombia"
angulo_de_venta = "Aumenta tu belleza natural"
tono = "Atractivo y coloquial (Medellín)"
tipo_de_video = "video corto para redes sociales"
duracion = 30  # Duración en segundos
print(f"Título: {titulos[0]}")
print(f"Copy: {copies[0]}")
print(f"Público objetivo: {publicos_objetivos[2]}")
print(f"País: {pais}")
print(f"Ángulo de venta: {angulo_de_venta}")
print(f"Tono: {tono}")
print(f"Tipo de video: {tipo_de_video}")
print(f"Duración: {duracion} segundos")

Título: Descubre el secreto para una mirada rejuvenecida
Copy: Desde la primera aplicación sentirás una frescura única mientras los ingredientes naturales trabajan en tu piel. Formulado sin alcohol ni parabenos, es ideal incluso para pieles sensibles. Olvídate de complicaciones con nuestro pago contra entrega y envío gratis.
Público objetivo: Piel sensible busca solucion natural
País: Colombia
Ángulo de venta: Aumenta tu belleza natural
Tono: Atractivo y coloquial (Medellín)
Tipo de video: video corto para redes sociales
Duración: 30 segundos


In [10]:
# Cargar el contenido del archivo JSON generado
agente = AgenteGuiones()
contenido = agente.generate_script(titulos[0], copies[0], pais, publicos_objetivos[2], angulo_de_venta, tono, tipo_de_video, duracion)
print(contenido)

¿Sueñas con una mirada fresca y rejuvenecida, pero tu piel sensible te limita?  

No más preocupaciones. Descubre el secreto para una mirada radiante, con ingredientes naturales que cuidan tu piel sin alcohol ni parabenos.  

Desde la primera aplicación sentirás una frescura única, mientras actúa suavemente. ¡Bolsas, ojeras y arrugas, fuera!  

Y lo mejor: pago contra entrega y envío gratis en toda Colombia. ¿Qué esperas?  

Dale a tu mirada el cuidado natural que se merece. ¡Pídelo ya y luce más joven sin complicaciones!


In [11]:
contenido = contenido.replace("\n\n", "\n")

In [12]:
print(contenido)

¿Sueñas con una mirada fresca y rejuvenecida, pero tu piel sensible te limita?  
No más preocupaciones. Descubre el secreto para una mirada radiante, con ingredientes naturales que cuidan tu piel sin alcohol ni parabenos.  
Desde la primera aplicación sentirás una frescura única, mientras actúa suavemente. ¡Bolsas, ojeras y arrugas, fuera!  
Y lo mejor: pago contra entrega y envío gratis en toda Colombia. ¿Qué esperas?  
Dale a tu mirada el cuidado natural que se merece. ¡Pídelo ya y luce más joven sin complicaciones!


In [13]:
# Guardar el contenido generado en un archivo .txt
with open("contenido_generado.txt", "w", encoding="utf-8") as txt_file:
    txt_file.write(contenido)

In [ ]:
import os
from elevenlabs.client import ElevenLabs

client = ElevenLabs(
  api_key=os.getenv("ELEVENLAB_API_KEY"),
)

audio_stream = client.text_to_speech.convert_as_stream(
    text=contenido,
    voice_id="qHkrJuifPpn95wK3rm2A",  # Rachel por ejemplo
    model_id="eleven_multilingual_v2",
    voice_settings={"speed": 1.1}  # Ajustar la velocidad a 1.1

)

# Guardar como archivo MP3
with open("voz_generada.mp3", "wb") as f:
    for chunk in audio_stream:
        if isinstance(chunk, bytes):
            f.write(chunk)


ApiError: status_code: 401, body: {'detail': {'status': 'quota_exceeded', 'message': 'This request exceeds your quota of 10000. You have 372 credits remaining, while 523 credits are required for this request.'}}

In [62]:
import requests

url = "https://api.heygen.com/v1/audio/upload"
headers = {
    "x-api-key": os.getenv("HEYGEN_API_KEY")
}
files = {
    "file": open("./voz_generada.mp3", "rb")
}

response = requests.post(url, headers=headers, files=files)
print(response.status_code)


404


In [63]:
import requests
import os

url = "https://api.heygen.com/v2/video/generate"
headers = {
    "X-Api-Key": os.getenv("HEYGEN_API_KEY"),
    "Content-Type": "application/json"
}

payload = {
    "video_inputs": [
        {
            "character": {
                "type": "avatar",
                "avatar_id": "Abigail_expressive_2024112501",  # Tu avatar personalizado
                "avatar_style": "normal"
            },
            "voice": {
                "type": "text",
                "input_text": "¡Hola! Bienvenido a Solvex, tenemos productos increíbles para ti.",
                "voice_id": "331f1cc78737475b946079cb3d2f5ffc",  # Cambia por el que prefieras
                "speed": 1.1
            }
        }
    ],
    "dimension": {
        "width": 1280,
        "height": 720
    }
}

response = requests.post(url, json=payload, headers=headers)
print(response.status_code)
print(response.json())



200
{'error': None, 'data': {'video_id': '68d0467ce55e476db65b8fcbccb964c5'}}


In [68]:
import requests
import time
import os

headers = {
    "X-Api-Key": os.getenv("HEYGEN_API_KEY"),
    "Accept": "application/json",
    "Content-Type": "application/json"
}

def obtener_video_url(video_id, headers):
    status_url = "https://api.heygen.com/v2/video_status"
    payload = {
        "video_id": video_id
    }

    while True:
        response = requests.post(status_url, json=payload, headers=headers)
        print("Status code:", response.status_code)
        print("Raw response:", response.text)

        try:
            data = response.json().get("data", {})
        except Exception as e:
            print("❌ Error al decodificar JSON:", e)
            break

        status = data.get("status")
        print("Estado del video:", status)

        if status == "completed":
            return data.get("video_url")
        elif status == "failed":
            raise Exception("La generación del video falló.")
        
        time.sleep(5)

# Uso real
video_id = "68d0467ce55e476db65b8fcbccb964c5"
video_url = obtener_video_url(video_id, headers)
print("🎥 Video listo:", video_url)




Status code: 404
Raw response: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

❌ Error al decodificar JSON: Expecting value: line 1 column 1 (char 0)
🎥 Video listo: None


In [ ]:
import requests
import time
import os

headers = {
    "X-Api-Key": os.getenv("HEYGEN_API_KEY"),
    "Accept": "application/json",
    "Content-Type": "application/json"
}

def obtener_video_url(video_id, headers):
    status_url = "https://api.heygen.com/v2/video_status"
    payload = {
        "video_id": video_id
    }

    while True:
        response = requests.post(status_url, json=payload, headers=headers)
        print("Status code:", response.status_code)
        print("Raw response:", response.text)

        try:
            data = response.json().get("data", {})
        except Exception as e:
            print("❌ Error al decodificar JSON:", e)
            break

        status = data.get("status")
        print("Estado del video:", status)

        if status == "completed":
            return data.get("video_url")
        elif status == "failed":
            raise Exception("La generación del video falló.")
        
        time.sleep(5)

# Uso real
video_id = "68d0467ce55e476db65b8fcbccb964c5"
video_url = obtener_video_url(video_id, headers)
print("🎥 Video listo:", video_url)




Status code: 404
Raw response: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

❌ Error al decodificar JSON: Expecting value: line 1 column 1 (char 0)
🎥 Video listo: None


In [43]:
import requests
import os

url = "https://api.heygen.com/v2/avatars"
headers = {
    "Accept": "application/json",
    "X-Api-Key": os.getenv("HEYGEN_API_KEY")
}

response = requests.get(url, headers=headers)
print(response.status_code)
print(response.json())


200
{'error': None, 'data': {'avatars': [{'avatar_id': 'Abigail_expressive_2024112501', 'avatar_name': 'Abigail (Upper Body)', 'gender': 'female', 'preview_image_url': 'https://files2.heygen.ai/avatar/v3/1ad51ab9fee24ae88af067206e14a1d8_44250/preview_target.webp', 'preview_video_url': 'https://files2.heygen.ai/avatar/v3/1ad51ab9fee24ae88af067206e14a1d8_44250/preview_video_target.mp4', 'premium': False, 'type': None, 'tags': None}, {'avatar_id': 'Abigail_standing_office_front', 'avatar_name': 'Abigail Office Front', 'gender': 'female', 'preview_image_url': 'https://files2.heygen.ai/avatar/v3/463208b6cad140d2b263535826838e3a_39240/preview_target.webp', 'preview_video_url': 'https://files2.heygen.ai/avatar/v3/463208b6cad140d2b263535826838e3a_39240/preview_video_target.mp4', 'premium': False, 'type': None, 'tags': None}, {'avatar_id': 'Abigail_standing_office_side', 'avatar_name': 'Abigail Office Side', 'gender': 'female', 'preview_image_url': 'https://files2.heygen.ai/avatar/v3/9886d46db7